<a href="https://colab.research.google.com/github/BigDesigner/Save-Youtube-videos-to-Google-Drive/blob/main/save-youtube-videos-to-google-drive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Requirements
First run code;

!pip install yt-dlp

!pip install ipywidgets


In [ ]:
!pip install yt-dlp
!pip install ipywidgets


# Connect Google Drive

In [ ]:
# Connect Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Save Youtube videos to Google Drive APP

**This project allows users to download YouTube videos directly to their Google Drive in the best available MP4 quality. Users can enter a YouTube URL and specify a folder name in their Google Drive for saving the downloaded videos.**

In [57]:
import os
import shutil
import yt_dlp
import ipywidgets as widgets
from IPython.display import display, clear_output

# Create the title
title = widgets.HTML(value="<h1>Save Youtube videos to Google Drive</h1>")

# Create input fields
link_label = widgets.HTML(value="<b>URL:</b>")
link_input = widgets.Text(placeholder='Enter YouTube URL...', layout=widgets.Layout(width='795px', height='40px'))

folder_label = widgets.HTML(value="<b>Folder:</b>")
folder_input = widgets.Text(placeholder='Default Folder MyDrive/Downloads or enter a new folder name.', layout=widgets.Layout(width='795px', height='40px'))

# Create the download button
download_button = widgets.Button(description='Download', layout=widgets.Layout(width='795px', height='40px'))

# Progress bar is initially hidden
progress_bar = widgets.FloatProgress(min=0.0, max=1.0, description='Progress:')
progress_bar.layout.display = 'none'
progress_label = widgets.Label(value="")

# Progress hook function to update the progress bar and label
def progress_hook(d):
    if d['status'] == 'downloading':
        total_bytes = d.get('total_bytes', 1)
        downloaded_bytes = d.get('downloaded_bytes', 0)
        progress_bar.value = downloaded_bytes / total_bytes if total_bytes else 0
        speed = d.get('speed', 0) / 1024 if d.get('speed') else 0
        eta = d.get('eta', 0)
        progress_label.value = f"Downloaded: {round(downloaded_bytes / 1024 / 1024, 2)} MB, Speed: {round(speed, 2)} KB/s, Remaining time: {eta} sec"
    elif d['status'] == 'finished':
        progress_bar.value = 1
        progress_label.value = "Download completed!"

# Function to handle the download process
def download_video(b):
    clear_output(wait=True)
    # Redisplay the page contents
    display(
        widgets.HBox([title], layout=widgets.Layout(justify_content='center', width='800px')),
        link_label, link_input,
        folder_label, folder_input,
        widgets.HBox([download_button], layout=widgets.Layout(justify_content='flex-end', width='800px')),
        progress_bar, progress_label
    )

    progress_bar.layout.display = 'block'
    progress_bar.value = 0
    progress_label.value = "Download starting..."

    url = link_input.value
    folder_path = folder_input.value

    if folder_path:
        folder_path = f'/content/drive/MyDrive/{folder_path}'
    else:
        folder_path = '/content/drive/MyDrive/Downloads'

    try:
        os.makedirs(folder_path, exist_ok=True)

        # Options for downloading the best quality MP4
        ydl_opts = {
            'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/mp4',
            'outtmpl': f'{folder_path}/%(title)s.%(ext)s',
            'noplaylist': False,
            'progress_hooks': [progress_hook]
        }

        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])
    except Exception as e:
        progress_label.value = f"An error occurred: {str(e)}"

    progress_bar.layout.display = 'none'

# Display the widgets in a box layout
display(widgets.VBox([
    widgets.HBox([title], layout=widgets.Layout(justify_content='center', width='800px')),
    link_label, link_input,
    folder_label, folder_input,
    widgets.HBox([download_button], layout=widgets.Layout(justify_content='flex-end', width='800px')),
    progress_bar, progress_label
], layout=widgets.Layout(width='800px', height='auto')))
download_button.on_click(download_video)
